In [1]:
import sys
sys.path.append('../scripts')

import yaml
import torch
from argparse import Namespace

from runner import Runner

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
with open('/home/jupyter/rfcx_submission/config/training_config.yaml', 'r') as f:
    args= yaml.load(f,  yaml.FullLoader)

In [4]:
args= Namespace(**args)

In [5]:
runner= Runner(device, args)

In [7]:
runner.set_data_loader()

training: number of docs : 378
evaluation: number of docs : 378


In [8]:
runner.set_model()

In [10]:
from fastai.learner import Learner
from fastai.callback.all import *
from fastai.optimizer import Adam


In [ ]:

import matplotlib.pyplot as plt


In [ ]:
def plot_spectrogram_to_numpy(spectrogram):
    spectrogram = spectrogram.transpose(1, 0)
    fig, ax = plt.subplots(figsize=(18, 3))
    im = ax.imshow(spectrogram, aspect="auto", origin="lower",cmap='magma')
    plt.colorbar(im, ax=ax)
    plt.xlabel("Frames")
    plt.ylabel("Channels")
    plt.tight_layout()

    fig.canvas.draw()


In [ ]:
plot_spectrogram_to_numpy(X[0][1].detach().numpy())

In [ ]:
plot_spectrogram_to_numpy(X[0][2].detach().numpy())